In [1]:
!pip3 install autokeras
!pip install tensorflow==2.1.0

     |████████████████████████████████| 71kB 3.7MB/s 
     |████████████████████████████████| 61kB 4.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=f9606fdbe02f0e5024302d9eec80dfdd5e9a30b4c6e15f4954e32da524f3274a
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=1f483c58d631a842b7682e51f68cf24f98f59ec3c96251500ed60eb01daccff5
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables
     |████████████████████████████████| 421.8MB 30kB/s 
     |████████████████████████████████| 3.9MB 28.5MB/s 
     |████████████████████████████████| 450kB 43.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=21961d41f4cc90d79f81c6af9991b9cc841ec1eff84c6645dc0f838

**Connect to Google Drive and import libraries**

In [5]:
from keras.preprocessing.image import ImageDataGenerator
import autokeras as ak
import keras
import numpy as np
from sklearn.metrics import classification_report
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Dowload data**

In [6]:
keras.utils.data_utils._extract_archive (
    '/content/gdrive/My Drive/Data/train.zip', path = 'train')
keras.utils.data_utils._extract_archive (
    '/content/gdrive/My Drive/Data/val.zip', path = 'val')
keras.utils.data_utils._extract_archive (
    '/content/gdrive/My Drive/Data/test.zip', path = 'test')

True

In [0]:
img_width, img_height = 48, 48
input_share = (img_width, img_height, 1)
epochs = 30
size_train = 20165
size_val = 3640
size_test = 4494
nb_train_samples = 20165
nb_validation_samples = 3640
nb_test_samples = 4494
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

In [8]:
datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = datagen.flow_from_directory(train_dir,
                                              target_size=(img_width, img_height),
                                              batch_size=size_train, class_mode='categorical',
                                              shuffle=1, color_mode='grayscale')
val_generator = datagen.flow_from_directory(val_dir,
                                            target_size=(img_width, img_height),
                                            batch_size=size_val, class_mode='categorical', color_mode='grayscale')
test_generator = datagen.flow_from_directory(test_dir,
                                             target_size=(img_width, img_height),
                                             batch_size=size_test, class_mode='categorical',
                                             color_mode='grayscale')
print(val_generator.class_indices)

Found 20165 images belonging to 7 classes.
Found 3640 images belonging to 7 classes.
Found 4494 images belonging to 7 classes.
{'Angry': 0, 'Fear': 1, 'Happy': 2, 'Neutral': 3, 'Sadness': 4, 'Scorn': 5, 'Surprise': 6}


In [9]:
x_train = train_generator[0][0]
y_train = train_generator[0][1]
x_test = test_generator[0][0]
y_test = test_generator[0][1]
x_val = val_generator[0][0]
y_val = val_generator[0][1]
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape,)

(20165, 48, 48, 1) (20165, 7) (4494, 48, 48, 1) (4494, 7) (3640, 48, 48, 1) (3640, 7)


In [0]:
# x_train = np.reshape(train_generator[0][0], (size_train, 48, 48))
# y_train = train_generator[0][1].astype('uint8')
# x_test = np.reshape(test_generator[0][0], (size_test, 48, 48))
# y_test = test_generator[0][1].astype('uint8')
# x_val = np.reshape(val_generator[0][0], (size_val, 48, 48))
# y_val = val_generator[0][1].astype('uint8')

In [0]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    # Normalize the dataset.
    normalize=False,
    # Do not do data augmentation.
    augment=False)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(inputs=input_node, outputs=output_node, max_trials=20)
clf.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size = 128, epochs=60)
best_model = clf.export_model()
try:
   best_model.save('/content/gdrive/My Drive/Models/model_self.h5')
except OSError:
    best_model.save('/content/drive/My Drive/Models/model_self.h5')

Train for 158 steps, validate for 29 steps
Epoch 1/60
158/158 [==============================] - 12s 76ms/step - loss: 1.9277 - accuracy: 0.1867 - val_loss: 1.8430 - val_accuracy: 0.3349
Epoch 2/60
158/158 [==============================] - 5s 34ms/step - loss: 1.8632 - accuracy: 0.2401 - val_loss: 1.8581 - val_accuracy: 0.2885
Epoch 3/60
158/158 [==============================] - 5s 34ms/step - loss: 1.8020 - accuracy: 0.2771 - val_loss: 1.7865 - val_accuracy: 0.3247
Epoch 4/60
158/158 [==============================] - 5s 34ms/step - loss: 1.7507 - accuracy: 0.3079 - val_loss: 1.6785 - val_accuracy: 0.3777
Epoch 5/60
158/158 [==============================] - 5s 34ms/step - loss: 1.7074 - accuracy: 0.3319 - val_loss: 1.6355 - val_accuracy: 0.3940
Epoch 6/60
158/158 [==============================] - 5s 34ms/step - loss: 1.6614 - accuracy: 0.3521 - val_loss: 1.6301 - val_accuracy: 0.3832
Epoch 7/60
158/158 [==============================] - 5s 34ms/step - loss: 1.6190 - accuracy: 0.37

Train for 158 steps, validate for 29 steps
Epoch 1/60
158/158 [==============================] - 51s 320ms/step - loss: 1.6368 - accuracy: 0.3587 - val_loss: 3.0744 - val_accuracy: 0.1574
Epoch 2/60
158/158 [==============================] - 37s 234ms/step - loss: 1.3134 - accuracy: 0.4941 - val_loss: 3.6916 - val_accuracy: 0.0945
Epoch 3/60
158/158 [==============================] - 37s 235ms/step - loss: 1.1130 - accuracy: 0.5785 - val_loss: 3.4646 - val_accuracy: 0.2022
Epoch 4/60
158/158 [==============================] - 37s 234ms/step - loss: 0.9773 - accuracy: 0.6309 - val_loss: 3.8379 - val_accuracy: 0.1720
Epoch 5/60
158/158 [==============================] - 38s 242ms/step - loss: 0.8569 - accuracy: 0.6789 - val_loss: 2.0567 - val_accuracy: 0.3390
Epoch 6/60
158/158 [==============================] - 37s 236ms/step - loss: 0.7612 - accuracy: 0.7160 - val_loss: 2.2221 - val_accuracy: 0.4137
Epoch 7/60
158/158 [==============================] - 37s 236ms/step - loss: 0.6497 - a

Train for 158 steps, validate for 29 steps
Epoch 1/60
158/158 [==============================] - 20s 124ms/step - loss: 1.9490 - accuracy: 0.1605 - val_loss: 1.9265 - val_accuracy: 0.2324
Epoch 2/60
158/158 [==============================] - 17s 107ms/step - loss: 1.9426 - accuracy: 0.1689 - val_loss: 1.9133 - val_accuracy: 0.2324
Epoch 3/60
158/158 [==============================] - 17s 107ms/step - loss: 1.9421 - accuracy: 0.1689 - val_loss: 1.9077 - val_accuracy: 0.2324
Epoch 4/60
158/158 [==============================] - 17s 107ms/step - loss: 1.9420 - accuracy: 0.1689 - val_loss: 1.9059 - val_accuracy: 0.2324
Epoch 5/60
158/158 [==============================] - 17s 107ms/step - loss: 1.9420 - accuracy: 0.1689 - val_loss: 1.9050 - val_accuracy: 0.2324
Epoch 6/60
158/158 [==============================] - 17s 106ms/step - loss: 1.9421 - accuracy: 0.1689 - val_loss: 1.9058 - val_accuracy: 0.2324
Epoch 7/60
158/158 [==============================] - 17s 105ms/step - loss: 1.9420 - a

Train for 158 steps, validate for 29 steps
Epoch 1/60
158/158 [==============================] - 169s 1s/step - loss: 3.2088 - accuracy: 0.1931 - val_loss: 1.9003 - val_accuracy: 0.2324
Epoch 2/60
158/158 [==============================] - 129s 816ms/step - loss: 2.3237 - accuracy: 0.1864 - val_loss: 1.9296 - val_accuracy: 0.1146
Epoch 3/60
158/158 [==============================] - 129s 817ms/step - loss: 2.1801 - accuracy: 0.2281 - val_loss: 1.9659 - val_accuracy: 0.1203
Epoch 4/60
158/158 [==============================] - 129s 817ms/step - loss: 2.0609 - accuracy: 0.2505 - val_loss: 9.9469 - val_accuracy: 0.2813
Epoch 5/60
158/158 [==============================] - 131s 830ms/step - loss: 1.8577 - accuracy: 0.3243 - val_loss: 1.7107 - val_accuracy: 0.3563
Epoch 6/60
158/158 [==============================] - 131s 827ms/step - loss: 1.7436 - accuracy: 0.3664 - val_loss: 1.6671 - val_accuracy: 0.3945
Epoch 7/60
158/158 [==============================] - 129s 818ms/step - loss: 1.5630